In [ ]:
# Step 1: Import required libraries
import pandas as pd
import numpy as np
import json
# Import the 're' module for regular expressions
import re  # This line is added to import the 're' module
from google.colab import files
uploaded = files.upload()


Saving compressed_data.csv.gz to compressed_data.csv.gz


In [ ]:
import pandas as pd
# Assuming your uploaded file is named "trades.csv"
data = pd.read_csv("compressed_data.csv.gz")
print(data.head())

              Port_IDs                                      Trade_History
0  3925368433214965504  [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1  4002413037164645377  [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2  3923766029921022977  [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3  3994879592543698688  [{'time': 1718678214000, 'symbol': 'ETHUSDT', ...
4  3926423286576838657  [{'time': 1718979615000, 'symbol': 'ETHUSDT', ...


In [10]:

# Step 3: Parse and clean data
# Assume 'Trade_History' is a nested column with JSON-like strings that need flattening.
if 'Trade_History' in data.columns:
    # Convert non-string values to valid JSON strings (e.g., empty dictionaries)
    data['Trade_History'] = data['Trade_History'].apply(lambda x: x if isinstance(x, str) else '{}')

    # Define a function to clean JSON strings
    def clean_json(json_string):
        # Replace single quotes with double quotes for keys
        json_string = re.sub(r"(\w+)\s*:", r'"\1":', json_string)
        # Replace single quotes with double quotes for values if they are not numbers or booleans
        json_string = re.sub(r":\s*'([^']*)'", r': "\1"', json_string)
        # Remove extra spaces and newlines
        json_string = re.sub(r'\s+', ' ', json_string)

        try:
            # Attempt to parse the cleaned JSON string
            json_data = json.loads(json_string)
            return json_string # Return the cleaned string if parsing is successful
        except json.JSONDecodeError as e:
            # Handle cases where cleaning didn't fix the JSON
            print(f"Error parsing JSON: {e}, Original string: {json_string}")  # Print error for debugging
            return '{}'  # Return an empty dictionary as a fallback

    # Apply the cleaning function to the 'Trade_History' column
    data['Trade_History'] = data['Trade_History'].apply(clean_json)

    # Use json.loads instead of eval for safer and more robust JSON parsing
    trade_history = pd.json_normalize(data['Trade_History'].apply(json.loads))

In [11]:
# Step 4: Group data by Port_ID for metric calculations

# Check if the 'side' column exists
if 'side' in data.columns:
    # Proceed to group by 'Port_IDs' if 'side' column exists
    grouped = data.groupby("Port_IDs")
else:
    # If 'side' column is missing, print an error and stop execution
    print("Error: 'side' column not found. Check your data and parsing logic.")
    grouped = None  # Set grouped to None to avoid further errors

# Ensure further code only runs if grouping was successful
if grouped is not None:
    print("Data successfully grouped by 'Port_IDs'. Proceeding with calculations.")
    # Add further processing logic here
else:
    print("Fix the missing 'side' column issue before continuing.")

if 'Port_IDs' not in data.columns:
    print("Error: 'Port_IDs' column not found. Please check your dataset.")

grouped = data.groupby("Port_IDs")

Error: 'side' column not found. Check your data and parsing logic.
Fix the missing 'side' column issue before continuing.


In [12]:

# Step 5: Define functions to calculate financial metrics
# ROI (Return on Investment)
def calculate_roi(trades):
    # Check if 'side' column exists before accessing it
    if 'side' in trades.columns:
        total_investment = trades.loc[trades['side'] == 'BUY', 'quantity'].sum()
    else:
        total_investment = 0  # Or handle it appropriately for your use case
    total_profit = trades['realizedProfit'].sum()
    roi = (total_profit / total_investment) * 100 if total_investment > 0 else 0
    return total_investment, total_profit, roi

# Sharpe Ratio
def calculate_sharpe_ratio(trades):
    returns = trades["realizedProfit"]
    if len(returns) > 1:
        return_mean = returns.mean()
        return_std = returns.std()
        return return_mean / return_std if return_std != 0 else np.nan
    return np.nan

In [13]:
print(data.columns)
grouped = data.groupby("Port_IDs")
if 'realizedProfit' not in data.columns:
    data['realizedProfit'] = data.get('profit', 0)  # Replace 'profit' with the actual column name.

Index(['Port_IDs', 'Trade_History'], dtype='object')


In [14]:
# Win Rate and Positions
def calculate_win_rate(trades):
    if 'realizedProfit' not in trades.columns:
        raise KeyError("'realizedProfit' column is missing in the DataFrame.")
    total_positions = len(trades)
    win_positions = len(trades[trades["realizedProfit"] > 0])
    win_rate = (win_positions / total_positions) * 100 if total_positions > 0 else 0
    return win_positions, total_positions, win_rate

# Maximum Drawdown (MDD)
def calculate_mdd(trades):
    if 'realizedProfit' not in trades.columns:
        raise KeyError("'realizedProfit' column is missing in the DataFrame.")
    cumulative_pnl = trades["realizedProfit"].cumsum()
    rolling_max = cumulative_pnl.cummax()
    drawdown = rolling_max - cumulative_pnl
    mdd = drawdown.max()
    return mdd


In [15]:

# Check and group data
if "Port_IDs" not in data.columns:
    raise KeyError("'Port_IDs' column is missing in the DataFrame.")

grouped = data.groupby("Port_IDs")

# Step 6: Calculate metrics for each account
roi_df = grouped.apply(lambda x: pd.Series(calculate_roi(x), index=["Total_Investment", "Total_Profit", "ROI"])).reset_index()
sharpe_df = grouped.apply(calculate_sharpe_ratio).reset_index()
sharpe_df.columns = ["Port_IDs", "Sharpe_Ratio"]
win_rate_df = grouped.apply(lambda x: calculate_win_rate(x)).reset_index()
win_rate_df.columns = ["Port_IDs", "Win_Positions_Tuple"]
win_rate_df[["Win_Positions", "Total_Positions", "Win_Rate"]] = pd.DataFrame(win_rate_df["Win_Positions_Tuple"].tolist(), index=win_rate_df.index)
win_rate_df = win_rate_df.drop(columns=["Win_Positions_Tuple"])
mdd_df = grouped.apply(calculate_mdd).reset_index()
mdd_df.columns = ["Port_IDs", "MDD"]


<ipython-input-15-411649a61a4d>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  roi_df = grouped.apply(lambda x: pd.Series(calculate_roi(x), index=["Total_Investment", "Total_Profit", "ROI"])).reset_index()
<ipython-input-15-411649a61a4d>:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sharpe_df = grouped.apply(calculate_sharpe_ratio).reset_index()
<ipython-input-15-411649a61a4d>:11: DeprecationWarning: D

In [16]:
# Step 7: Combine all metrics into one DataFrame
metrics_df = roi_df.merge(sharpe_df, on="Port_IDs").merge(win_rate_df, on="Port_IDs").merge(mdd_df, on="Port_IDs")

# Step 8: Normalize metrics and calculate weighted score for ranking
weights = {
    "ROI": 0.4,
    "PnL": 0.3,
    "Sharpe_Ratio": 0.2,
    "Win_Rate": 0.1,
    "MDD": -0.1,
}

for col in ["ROI", "Total_Profit", "Sharpe_Ratio", "Win_Rate", "MDD"]:
    if col == "MDD":
        metrics_df[col + "_Normalized"] = 1 - (metrics_df[col] / metrics_df[col].max())
    else:
        metrics_df[col + "_Normalized"] = metrics_df[col] / metrics_df[col].max()

metrics_df["Weighted_Score"] = (
    metrics_df["ROI_Normalized"] * weights["ROI"]
    + metrics_df["Total_Profit_Normalized"] * weights["PnL"]
    + metrics_df["Sharpe_Ratio_Normalized"] * weights["Sharpe_Ratio"]
    + metrics_df["Win_Rate_Normalized"] * weights["Win_Rate"]
    + metrics_df["MDD_Normalized"] * weights["MDD"]
)

metrics_df["Rank"] = metrics_df["Weighted_Score"].rank(ascending=False)

# Step 9: Extract the top 20 accounts
top_20_accounts = metrics_df.sort_values(by="Rank").head(20)

# Step 10: Save results to CSV files
metrics_df.to_csv("all_metrics.csv", index=False)
top_20_accounts.to_csv("top_20_accounts.csv", index=False)

# Print file paths for download
print("Full metrics saved as 'all_metrics.csv'")
print("Top 20 accounts saved as 'top_20_accounts.csv'")

Full metrics saved as 'all_metrics.csv'
Top 20 accounts saved as 'top_20_accounts.csv'
